In [11]:
# Relevant imports
import torch
import sys
import matplotlib.pyplot as plt
import numpy as np
from kan import KAN
import pysindy as ps
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm

# Set the seed for reproducibility
torch.manual_seed(42)

# Add the parent directory of the script (i.e., project/) to sys.path
sys.path.append('../../utils')
from upinn import UPINN, NullWork
from architectures import FNN, StackedNN
from utils import compute_grad
from NavierStokesData import NavierStokesData

In [12]:
class NavierStokesUPINN(UPINN):
    
    def F_input(self, Z, U):
        
        psi = U[:, 0:1]
        
        U_z = compute_grad(psi, Z)

        psi_x = U_z[:, 0:1]
        psi_y = U_z[:, 1:2]

        u = psi_y
        v = -psi_x

        return torch.cat([Z, u, v], dim=1)


In [13]:
hidden = [64] * 8

u = FNN(
    dims=[3, *hidden, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.Identity(),
)

F = FNN(
    dims=[5, *hidden, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.Identity(),
)

In [14]:
# u1 = FNN(
#     dims=[3, *hidden, 1],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.Identity(),
# )

# u2 = FNN(
#     dims=[3, *hidden, 1],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.Identity(),
# )

# u = StackedNN([u1, u2])

# F1 = FNN(
#     dims=[5, *hidden, 1],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.Identity(),
# )

# F2 = FNN(
#     dims=[5, *hidden, 1],
#     hidden_act=torch.nn.Tanh(),
#     output_act=torch.nn.Identity(),
# )

# F = StackedNN([F1, F2])

In [15]:
model = NavierStokesUPINN(u, N=NullWork(), F=F)

[Info]: Initializing UPINN model


In [16]:
name = 'ns-upinn-with-pressuredata-FNN64-sa02-RAD-alt-2000_new'
model.load(name, 'models')

[Error]: Model not found.


### Sample data from UPINN

In [17]:
x = torch.linspace(2, 8, 50)
y = torch.linspace(-2, 2, 50)
t = torch.linspace(0, 20, 50)
domain_shape = (len(x), len(y), len(t))

X, Y, T = torch.meshgrid(x, y, t)

Z = torch.stack([X, Y, T], dim=3).reshape(-1, 3)
Z.requires_grad_(True)

U_pred = model.u(Z)
F_pred = model.F(model.F_input(Z, U_pred))

Psi = U_pred[:, 0:1]
Psi_Z = compute_grad(Psi, Z)
Psi_X = Psi_Z[:, 0:1]
Psi_Y = Psi_Z[:, 1:2]

U_pred = Psi_Y.detach().numpy().reshape(*domain_shape)
V_pred = -Psi_X.detach().numpy().reshape(*domain_shape)

u = np.stack([U_pred, V_pred], axis=3)

u_dot = F_pred.reshape(*domain_shape, 2).detach().numpy()

### Define function library

In [18]:
X, Y = np.meshgrid(x, y)
spatial_grid = np.asarray([X, Y]).T
t = t.detach().numpy()

library_functions = [
    lambda x: x,
]
library_function_names = [
    lambda x: x,
]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=2,
    spatial_grid=spatial_grid,
    # include_bias=True,
    is_uniform=True,
    include_interaction=True,
    # periodic=True
)

In [10]:
cv_folds = TimeSeriesSplit(n_splits=5)
scores = []
min_score = []
max_score = []
taus = np.linspace(0.1, 2, 20)
for t_ in taus:
    optimizer = ps.SR3(
        threshold=t_,
        max_iter=100000,
        thresholder="l1",
        normalize_columns=True,
    )
    model_sindy = ps.SINDy(feature_library=pde_lib, optimizer=optimizer, feature_names=['u', 'v'])
    cv_scores = []
    for train_idx, test_idx in tqdm(cv_folds.split(t)):
        model_sindy.fit(x=u[:, :, train_idx, :], t=t[train_idx], x_dot=u_dot[:, :, train_idx, :])
        cv_scores.append(model_sindy.score(x=u[:, :, test_idx, :], t=t[test_idx]))
    scores.append(np.mean(cv_scores))
    min_score.append(np.min(cv_scores))
    max_score.append(np.max(cv_scores))
    print(f'Threshold: {t_},\n Mean Score: {scores[-1]}')

best_tau = taus[np.argmin(scores)]

5it [00:27,  5.46s/it]


Threshold: 0.1,
 Mean Score: -0.009764576160193305


5it [00:15,  3.07s/it]


Threshold: 0.2,
 Mean Score: -0.008607790960123828


4it [00:08,  2.00s/it]


KeyboardInterrupt: 

In [20]:
optimizer = ps.SR3(
        threshold=50,
        thresholder="l1",
        max_iter=100000,
        normalize_columns=True,
    )
model_sindy = ps.SINDy(feature_library=pde_lib, optimizer=optimizer, feature_names=['u', 'v'])
model_sindy.fit(u, x_dot=u_dot, t=t)
model_sindy.print()

(u)' = -14.702 u
(v)' = 0.000
